# 【模板】网络最大流

## 题目描述

如题，给出一个网络图，以及其源点和汇点，求出其网络最大流。

## 输入格式

第一行包含四个正整数 $n,m,s,t$，分别表示点的个数、有向边的个数、源点序号、汇点序号。

接下来 $m$ 行每行包含三个正整数 $u_i,v_i,w_i$，表示第 $i$ 条有向边从 $u_i$ 出发，到达 $v_i$，边权为 $w_i$（即该边最大流量为 $w_i$）。

## 输出格式

一行，包含一个正整数，即为该网络的最大流。

## 样例 #1

### 样例输入 #1

```
4 5 4 3
4 2 30
4 3 20
2 3 20
2 1 30
1 3 40
```

### 样例输出 #1

```
50
```

## 提示

#### 样例输入输出 1 解释

 ![](https://cdn.luogu.com.cn/upload/pic/2262.png) 

题目中存在 $3$ 条路径：

- $4\to 2\to 3$，该路线可通过 $20$ 的流量。
- $4\to 3$，可通过 $20$ 的流量。
- $4\to 2\to 1\to 3$，可通过 $10$ 的流量（边 $4\to 2$ 之前已经耗费了 $20$ 的流量）。

故流量总计 $20+20+10=50$。输出 $50$。

---

#### 数据规模与约定

- 对于 $30\%$ 的数据，保证 $n\leq10$，$m\leq25$。
- 对于 $100\%$ 的数据，保证 $1 \leq n\leq200$，$1 \leq m\leq 5000$，$0 \leq w\lt 2^{31}$。

# dinic算法
参考[题解](https://www.luogu.com.cn/problem/solution/P3376)

In [25]:
from collections import deque
from typing import List
import numpy as np

class edge:
    def __init__(self,to:int,w:int,nxt:int):
        self.to=to
        self.w=w
        self.nxt=nxt
class dinic:
    def __init__(self,n:int,m:int,s:int,t:int,edges:List[List[int]]):
        self.n=n
        self.m=m
        self.s=s
        self.t=t
        self.cnt=0
        self.ans=0
        self.e=[edge(0,0,0)]
        self.head=np.zeros(n+5,dtype=int)
        self.dep=np.zeros(n+5)
        for i in edges:#链式前向星存图
            u,v,w=i[0],i[1],i[2]
            self.add(u,v,w)
            self.add(v,u,0)
    def solve(self)->int:
        while self.bfs()!=0:#当汇点的深度为初始值（0），说明已经没有增广路径
            self.ans+=self.dfs(self.s,1e18)
        return self.ans
      
        
    def add(self,u:int,v:int,w:int):
        self.cnt+=1
        self.e.append(edge(to=v,w=w,nxt=self.head[u]))
        self.head[u]=self.cnt
    def bfs(self)->int:#根据残量网络重新分层，并返回汇点所在的层数（可用于判断是否还有增广路径）
        self.dep[:]=0
        q=deque([])
        q.append(self.s)
        self.dep[self.s]=1
        while q:
            tmp=q[0]
            i=self.head[tmp]
            while i:
                tt=self.e[i].to
                if self.dep[tt]==0 and self.e[i].w>0:
                    self.dep[tt]=self.dep[tmp]+1
                    q.append(tt)
                i=self.e[i].nxt
            q.popleft()
        return self.dep[self.t]
    
    #现在位于now，能够允许的最大流量为maxn,返回实际从该点流出的流量
    def dfs(self,now:int,maxn:int):
        if now==self.t:
            return maxn
        ret=0
        i=self.head[now]
        while i and maxn:
            tt=self.e[i].to
            if self.dep[tt]==self.dep[now]+1 and self.e[i].w>0:#只尝试将剩余的流量给下一层的点
                a=self.dfs(tt,min(maxn,self.e[i].w))
                ret+=a
                maxn-=a
                if i%2==1:
                    self.e[i].w-=a
                    self.e[i+1].w+=a
                else:
                    self.e[i].w-=a
                    self.e[i+1].w+=a
            i=self.e[i].nxt
        if ret==0:
            self.dep[now]=0
        return ret
                
# n,m,s,t=4,5,4,3
# edges=[[4,2,30],[4,3,20],[2,3,20],[2,1,30],[1,3,40]]
# print(dinic(n,m,s,t,edges).solve())

n,m,s,t= map(int, input().split())
edges=[]
for i in range(m):
    edges.append(list(map(int, input().split())))
print(dinic(n,m,s,t,edges).solve())

4 5 4 3
4 2 30
4 3 20
2 3 20
2 1 30 
1 3 40
50
